## データセットのフレーム可視化
- rellisがscannetに比べて学習時のlossが高すぎる
- frame_selectionの時点でおかしい説？

In [1]:
import open3d

In [1]:
import os

In [2]:
import pickle
import torch
import open3d as o3d
from natsort import natsorted
import os
from glob import glob
from scipy.spatial.transform import Rotation as R
import numpy as np

import numpy as np
# from open3d import JVisualizer

In [ ]:
o3d.__version__

In [4]:
def load_pkl(path):
    with open(path, 'rb') as f:
        pkl = pickle.load(f)
    return pkl

# rellis(einrul)
rellis_root = '/Users/yukiya/ws/dataset/EINRUL'
sec = os.listdir(rellis_root)
pose_path = os.path.join(rellis_root, sec[0], 'poses.pkl')
pcd_path = natsorted(glob(os.path.join(rellis_root, sec[0], "**/*.pcd"), recursive=True))

# scannet
# pose_path = '../scene/scannet/scene0000_00/final_pose.pkl'
# final_pcd_path = '../scene/scannet/scene0000_00/final_pcd.pcd'

# rellis-3d
# pose_path = '../scene/rellis-3d/0/final_pose.pkl'
# final_pcd_path = '../scene/rellis-3d/0/final_pcd.pcd'

with open(pose_path, 'rb') as f:
    poses = pickle.load(f)

print(f"available {len(pcd_path)} pcd files and {len(poses)} poses.")

available 2059 pcd files and 2059 poses.


In [6]:
def transform_pcd(pcd, quaternion, translation):
    # クォータニオンから回転行列を生成
    rotation_matrix = R.from_quat([quaternion['x'], quaternion['y'], quaternion['z'], quaternion['w']]).as_matrix()

    # 変換行列を作成
    transform_matrix = np.eye(4)
    transform_matrix[:3, :3] = rotation_matrix
    transform_matrix[:3, 3] = [translation['x'], translation['y'], translation['z']]

    # 点群に変換を適用
    pcd.transform(transform_matrix)

    return pcd

def transform_pcd_from_dict(pcd, trans_dict):
    return transform_pcd(pcd, trans_dict['rotation'], trans_dict['position'])

In [5]:
loaded_pose = load_pkl(pose_path)
pcd = o3d.io.read_point_cloud(final_pcd_path)
# pcd_list = [o3d.io.read_point_cloud(final_pcd_path)

o3d.visualization.draw_geometries([pcd])

NameError: name 'final_pcd_path' is not defined

In [9]:
split = 100

pcd_list = [o3d.io.read_point_cloud(pcd_path[idx]) for idx in range(0, len(pcd_path), split)]
pose_list = [poses[idx] for idx in range(0, len(pcd_path), split)]

transformed_list = [transform_pcd_from_dict(pcd_list[idx], pose_list[idx]) for idx in range(len(pose_list))]

In [10]:
o3d.visualization.draw_geometries(transformed_list)
# draw(transformed_list)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
